In [1]:
%matplotlib widget
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import os
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
from ipywidgets import fixed, Layout, Button, Box
import scipy.optimize as spopt 
import io
import kmc.tools
#plt.style.use(['labplot', 'labplot_note'])

###################
#Preliminary setup#
###################

#getting the path file to dump txts after
try:
    with open('pathfile.txt','r') as p:
        for line in p:
            path_bash = line
except:
    path_bash = ''
#files = [i for i in os.listdir('.') if '.txt' in i]


###Buttons to be used####
#read file button
run_but = widgets.ToggleButton(
    value=False,
    description='Read File',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

#dropdown = widgets.Dropdown(options=files, values=files[0],description='File name',disabled=False)

#file manager
dropdown = widgets.FileUpload(
    accept='.txt',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True  # True to accept multiple files upload else False
)


#button to download the data from histogram
dump_but_hist = widgets.ToggleButton(
    value=False,
    description='Export Data',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    )
##########################

In [12]:
def filter(data,kwargs):
    #getting the restrictions to the dataset from the buttons
    thekeys = list(set([i[:-1] for i in kwargs.keys()]))
    #filtering the dataframe to leave only the dataset of interest
    data_filtered = []
    for key in thekeys:
        mats,energy,deaths = [kwargs[i] for i in kwargs.keys() if key in i]
        data_filtered.append(kmc.tools.filter_per_particle(data,key,deaths,mats,energy))    
    data_filtered =  pd.concat(data_filtered)
    return data_filtered

#####################################
# widgets are included through two functions:
# XXX_widget : recieves the variables from the interface's buttons and treat it to become suitable for analysis
# XXX_func   : carries out the analysis and visualization
#####################################


#### TRPL #######################
def trpl_func(dataframe,bin_num,tot_len,dump_but,legend,area_display):
    plt.figure(1)
    plt.clf()
    fluorescence = dataframe['Time'].to_numpy(float)/1000
    hist, bins = kmc.tools.trpl(fluorescence,bin_num=bin_num) 
    
    plt.xlabel("Time (ns)")          
    plt.ylabel("Intensity (a.u.)")   
    plt.ylim([0,1.05])
    
    plt.plot(bins,hist/max(hist), label=legend)
    avg_lf = np.mean(fluorescence)
    cutoff = np.max(fluorescence)
    area = 0
    for h in range(len(hist)-1):
        area += hist[h]*(bins[h+1]-bins[h])
        if area >= area_display/100:
            cutoff = bins[h]
            break

    plt.xlim([0,cutoff])
    print(f'The average lifetime is: {avg_lf:.2f} ns')
    print(f'We are using {100*len(fluorescence)/tot_len:.2f} % of the total data')
    
    plt.legend()
    plt.show()
    
    if dump_but:
        hist_data = np.column_stack([bins, hist/max(hist)])
        np.savetxt(path_bash+'/trpl.txt'     , hist_data, fmt=['%f','%f'])
    
def trpl_widget(data,tot_len,**kwargs):   
    data_filtered =  filter(data,kwargs)
    
    try:
        bin_slider = widgets.IntSlider(
        value=100,
        max=300,
        min=2,
        step=1,
        description='Bins:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
        )
        
        area_slider = widgets.IntSlider(
        value=50,
        max=100,
        min=1,
        step=1,
        description='Fraction:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
        )

        dump_but = widgets.ToggleButton(
            value=False,
            description='Export Data',
            disabled=False,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Description',
            icon='check'
        )

        #gettin which processes were selected to the trpl calculation
        legend = []
        for i in kwargs.keys():
            if '2' in i:
                try:
                    legend.append(kwargs[i][0])
                except:
                    pass
        legend = ', '.join(legend).capitalize() #capitalize(' '.join(legend))
        #widgets.interact(trpl_func,dataframe=fixed(data_filtered),bin_num = bin_slider,tot_len=fixed(tot_len),dump_but=dump_but,legend=fixed(legend),area_display=area_slider)
        wid = widgets.interactive(trpl_func,dataframe=fixed(data_filtered),bin_num = bin_slider,tot_len=fixed(tot_len),dump_but=dump_but,legend=fixed(legend),area_display=area_slider)
        box  = widgets.HBox([wid.children[0],wid.children[2],wid.children[1]])
        bbox = widgets.VBox([box,wid.children[-1]])
        display(bbox)        
    except:
        print('error')
        pass
###################################
######### histogram ###############
def make_hist_plot(histogram_data):
    mortes = histogram_data['CausaMortis'].unique()
    nums = []
    for morte in mortes:
        n = histogram_data[histogram_data.CausaMortis == morte].shape[0]
        nums.append(n)
    nums = np.array(nums)    
    mortes = [m.upper() for m in mortes]
    plt.figure(2)
    plt.clf()
    plt.bar(mortes, height=100*nums/np.sum(nums))
    plt.ylabel('Distribution (%)')
    plt.show()
    
def histogram_widget(data,tot_len,**kwargs):
    data_filtered =  filter(data,kwargs)

    make_hist_plot(data_filtered)
    
    #organize the boxes by particle type
    #def organize_boxes(num_ele,widget_int):
    #    labels = ['Singlet','Triplet', 'Hole', 'Electron']
    #    n_boxes = int(len(widget_int.children)/num_ele)
    #    #print(n_boxes)
    #    v = widgets.VBox([widgets.Label('')])
    #    for k in range(n_boxes):
    #        rang = range(num_ele*k,(num_ele)*(k+1)-1)
    #        print(labels[k],rang)
    #        h = widgets.HBox([widget_int.children[i] for i in rang])
    #        v = widgets.VBox([v,widgets.Label(labels[k]),h])
    #    v = widgets.VBox([v,widget_int.children[-1]])
    #    return v
                        
###################################
def data_plot(dataframes,sel,sli):
    for data in dataframes:
        if data.name in sel:
            display(data.name)
            display(data.sort_index().reset_index(drop=True).iloc[sli[0]:sli[1]])
def data_widget(dataframes,**kwargs):
    df_filtered_list = [filter(data,kwargs) for data in dataframes]
    
    for i in range(len(df_filtered_list)):
        df_filtered_list[i].name = dataframes[i].name
        
    selected_df = widgets.SelectMultiple(
        options =[data.name for data in dataframes],
        value   =[[data.name for data in dataframes][0]],
        description='Logs loaded:',
        disabled=False
    )
    biggest_len = min([len(data) for data in df_filtered_list])
    slider = widgets.IntRangeSlider(
        value=[0, biggest_len],
        min=0,
        max=biggest_len,
        step=biggest_len/30,
        description='Range:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
        #layout=widgets.Layout(height='100%')
    )
    wid  = widgets.interactive(data_plot,dataframes=fixed(df_filtered_list),sel=selected_df,sli=slider)              
    box  = widgets.HBox([wid.children[0],wid.children[1]])
    bbox = widgets.VBox([box,wid.children[-1]])
    display(bbox)
    
def spec_plot(data,gran):
    engs = data['Energy'].to_numpy()

    hist, bins = kmc.tools.spectrum(engs,gran)

    plt.figure(3)
    plt.clf()
    plt.plot(bins, hist)
    plt.ylabel('Spectra')
    plt.xlabel('Energy (eV)')
    plt.ylim(bottom=0)
    plt.show()

def spec_widget(data,tot_len,**kwargs):
    data_filtered =  filter(data,kwargs)

    gran_slider = widgets.FloatSlider(
    value=0.01,
    max=0.2,
    min=0.01,
    step=0.01,
    description='Granularity',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
    )

    widgets.interact(spec_plot,data=fixed(data_filtered),gran=gran_slider)

def diff_plot(data,gran):
    dx = data['DeltaX'].to_numpy()/10
    dy = data['DeltaY'].to_numpy()/10
    dz = data['DeltaZ'].to_numpy()/10
    ld = dx*dx+dy*dy+dz*dz
    ds = np.sqrt(ld)
    ld = np.sqrt(np.mean(ld))
    histx, binsx = kmc.tools.spectrum(dx,gran)
    histy, binsy = kmc.tools.spectrum(dy,gran)
    histz, binsz = kmc.tools.spectrum(dz,gran)
    hists, binss = kmc.tools.spectrum(ds,gran)

    fig,ax = plt.subplots(1,3,figsize=(11,4))
    ax[0].plot(binsx, histx/max(histx))
    ax[1].plot(binsy, histy/max(histy))
    ax[2].plot(binsz, histz/max(histz))
    ax[0].set_xlabel('$\Delta x$ (nm)')
    ax[1].set_xlabel('$\Delta y$ (nm)')
    ax[2].set_xlabel('$\Delta z$ (nm)')
    for a in ax:
        a.set_ylabel('Distribution')
        a.set_ylim(bottom=0)
    plt.tight_layout()
    plt.show()
    
    fig,ax = plt.subplots(figsize=(11,4))
    ax.plot(binss, hists/max(hists),label=f'L$_D={ld:.1f}$ nm')
    ax.vlines(ld,0,1,color='red',linestyle='dashed')
    ax.set_xlabel('$L_D$ (nm)')
    ax.set_ylabel('Distribution')
    ax.set_xlim(left=0)
    ax.set_ylim(bottom=0)
    ax.legend(loc='best')
    plt.show()

def diff_widget(data,tot_len,**kwargs):
    data_filtered =  filter(data,kwargs)

    gran_slider2 = widgets.FloatSlider(
    value=1,
    max=5,
    min=0.1,
    step=0.5,
    description='Granularity',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
    )

    widgets.interact(diff_plot,data=fixed(data_filtered),gran=gran_slider2)
    
#core function
def main(file_name,run_button):
    #print('the file '+ list(file_name.keys())[0] + 'was loaded!')
    if run_button: #if 'Read file' is pressed
        name = list(file_name.keys())[0]
        
        input_list = list(file_name.keys())
        dataframes = []
        for name in input_list:
            data = file_name[name]['content']
            data = io.StringIO(data.decode('utf-8'))
            data = pd.read_csv(data)
            data = data[data.Time != 'END'] #removes the end of round lines
            data.name = name                #labeling the dataframes
            dataframes.append(data)
            
        data = dataframes[0] #input to be used as reference
        tot_len    = np.shape(data)[0]
        species    = np.unique(np.array([item for item in data['Type'].unique() for data in dataframes]))
        max_energy = np.amax([np.amax(data['Energy']) for data in dataframes])
        min_energy = np.amin([np.amin(data['Energy']) for data in dataframes])
        
        ########################################    
        #defining widgets after pre load process
        #for each particle there are two sets of widgets: 
        # 1) the ones attached to the table : [energy , mats,   deaths, status] 
        # 2) the ones attached to the widget.interactive : [energy2, mats2, deaths2, status2] 
        # Comunication between them is mediated by the report_ie_change function
        
        obserb_dics, obstab, names,boxes = [], [], [], [] 
        for part in species:
            #could extend for the other dataframes. Not sure if it is worth
            data_part     = kmc.tools.filter_data_list(data,'Type',[part])
            unique_mats   = data_part['Location'].unique()
            unique_death  = data_part['CausaMortis'].unique()
            #unique_status = data_part['Status'].unique()
            wids          = kmc.tools.get_widgets_per_particle(min_energy,max_energy,unique_mats,unique_death)
            wids2         = kmc.tools.get_widgets_per_particle(min_energy,max_energy,unique_mats,unique_death)
            names.extend([part+str(i) for i in range(len(wids))])
            obserb_dics.extend(wids)
            obstab.extend(wids2)
            boxes.append(kmc.tools.make_boxes(wids2))
            
        
        kw,avatar = {}, {}
        for i in range(len(names)):
            kw[names[i]]     = obserb_dics[i]
            avatar[names[i]] = obstab[i]

        
        ###########################
        #initializing table
        output       = widgets.Output()
        tab          = widgets.Tab()
        tab.children = boxes
        #setting up the titles of the table
        for i in range(len(species)):
            tab.set_title(i,species[i].upper())
        display(tab,output)
        
        w_data = widgets.interactive(data_widget,dataframes=fixed(dataframes),**kw)
        #w_data = widgets.interactive(data_widget,data=fixed(data),**kw)

        #Calling Histogram INTERACTIVE INTERFACE
        w_hist = widgets.interactive(histogram_widget,data=fixed(data),tot_len=fixed(tot_len),**kw)

        
        #Calling TRPL INTERACTIVE INTERFACE
        w_trpl = widgets.interactive(trpl_widget,data=fixed(data),tot_len=fixed(tot_len),**kw)

        w_spec = widgets.interactive(spec_widget,data=fixed(data),tot_len=fixed(tot_len),**kw)
        
        w_diff = widgets.interactive(diff_widget,data=fixed(data),tot_len=fixed(tot_len),**kw)

        accordion = widgets.Accordion(children=[w_data,w_trpl,w_hist,w_spec,w_diff], selected_index=None)
        accordion.set_title(0, 'DATA')
        accordion.set_title(1, 'TIME RESOLVED PHOTOLUMINESCENCE')
        accordion.set_title(2, 'HISTOGRAM')
        accordion.set_title(3, 'SPECTRA')
        accordion.set_title(4, 'DIFFUSION')
        accordion.set_title(5, 'HEATMAP')
        display(accordion)
        
        #this function updates the atribute value upon changes in the widget. Must remain defined inside of main()
        def report_ie_change(change):#,accordion,attrib,indx):
            with output:    
                index = list(avatar.values()).index(change.owner)        
                list(kw.values())[index].value = change.new

        for wid in avatar.values():
            wid.observe(report_ie_change,names=['value'])
        
        #needed because by default widget.interactive pops up the widgets. Tuning them invisible here..
        for indx in range(len(w_trpl.children)-1):
            w_trpl.children[indx].close()
        
        
    else:
        pass 

In [13]:
###################################################
#Initializing main function and displaying widgets
###################################################
display(widgets.HTML(value = r'<p style="font-size:24px"><b>DASHBOARD KMC</b></p>'))
i = widgets.interactive(main, file_name=dropdown,run_button=run_but);
h = widgets.HBox([i.children[0],i.children[1]])
v = widgets.VBox([h,i.children[2]])
display(v)

HTML(value='<p style="font-size:24px"><b>DASHBOARD KMC</b></p>')